In [ ]:
# #ephemeris sim
# using LinearAlgebra
# #make sure to cite this package
# using SPICE
# using Downloads: download 
# using DifferentialEquations
# using Plots

In [10]:
using LinearAlgebra
using SPICE
using DifferentialEquations
using Plots


In [ ]:
# using Pkg
# Pkg.activate(".")

In [ ]:
using DelimitedFiles
xtraj  = readdlm("xtraj.txt", '\t', Float64, '\n')

#in days
time_steps = readdlm("time.txt", '\t', Float64, '\n')
#period 
#T_periodic_n = 3.414975413662902


T_periodic_n = 3.414975410587678

In [ ]:
# Load leap seconds kernel
#furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/naif0012.tls")

#laptop 
furnsh("/home/fausto/naif0012.tls")

In [ ]:
# Convert the calendar date to ephemeris seconds past J2000

#was 2018
#et = utc2et("2018-02-06T20:45:00")

et = 0

In [ ]:
# Load a planetary ephemeris kernel
#furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/de440.bsp")

#laptop
# furnsh("/home/fausto/de440.bsp")

In [ ]:
#gravitational parameters for the bodies (km3/s2)

#moon 
μ_m = 4902.80058214776

#earth
μ_e = 398600.432896939

#sun
μ_s = 132712440017.987

In [ ]:
#units used to normalize
#also in km and seconds

#distance
l_star = 385692.50

#time
t_star = 377084.152667038

#in km
lunar_radius = 1738.2 

In [ ]:
#get the jacobian that maps from rotating frame to inertial frame
function get_transformation(time)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    moon_state = spkezr("moon",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_moon = moon_state[1:3]

    #velocity of the moon relative to Earth
    v_moon = moon_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_moon/norm(r_moon)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_moon, v_moon)/norm(cross(r_moon, v_moon))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)

    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    C = [x̃ ỹ z̃]

    #instantanous angular velocity
    #θ_dot = norm(cross(r_moon, v_moon))/(norm(r_moon)^2)

    #θ_dot = (cross(r_moon, v_moon)/(norm(r_moon)^2))[3]

    θ_dot = 2*pi/2.361e6


    return C, θ_dot

end

In [ ]:
function cr3bp_to_ephem(x, t)

    time = et + t

    #get transformation takes in time wrt et
    C, θ_dot = get_transformation(time)

    C̄ = [θ_dot.*C[:,2] -θ_dot.*C[:,1]]

    #transformation matrix
    C̃ = [C zeros(3,3); C̄ zeros(3) C]

    #apply the rotation to get the state in the inertial frame
    x_ephem = C̃*x

    return x_ephem

end

In [ ]:
function ephem_to_cr3bp(x, t)

    time = et + t

    #get transformation takes in time wrt et
    C, θ_dot = get_transformation(time)

    C̄ = [θ_dot.*C[:,2] -θ_dot.*C[:,1]]

    #transformation matrix
    C̃ = [C zeros(3,3); C̄ zeros(3) C]

    #get the inverse transformation matrix

    C_inv = inv(C̃)
    #C_inv = C̃'
    #apply the rotation to get the state in the inertial frame
    x_cr3bp = C_inv*x

    return x_cr3bp

end

In [ ]:
function ephem_dynamics_scaled!(du, u, p, t)
    
    #scaled version
    du[1:6] = scaled_ephemeris_dynamics(u[1:6], t)
    
    
end

In [ ]:
#gets the state and for the entire solution

function get_state(solution)
    
    N = size(solution.u)[1]

    all_states = zeros(6, N)

    for i=1:N
        all_states[:,i] = solution.u[i][1:6]
    end
    
    #all states and all stm are functions of t
    #solution.t is the time
    return all_states
end

In [ ]:
function just_dynamics_integrate_scaled(x_0, period)
    
    #integrate from 0 to period
    tspan = (0.0, period)
    prob = ODEProblem(ephem_dynamics_scaled!, x_0, tspan)
    sol = solve(prob, TsitPap8(), abstol=1e-12, reltol=1e-12)
    
    return sol
    
end

In [ ]:
#the state is in km and s as well as the time

function ephemeris_model_EarthMoon(x, t)

    xdot = zeros(6)

    xdot[1:3] = x[4:6]

    #position of the spacecraft relative to the earth 
    r_qi = x[1:3]

    time = et + t

    #get positions of moon and sun relative to earth 
    rqj_moon = spkpos("moon", time, "J2000", "none", "earth")[1]
    

    #println("position of the satellite: ", r_qi)

    #println("position of the moon: ", rqj_moon)

    rij_moon = rqj_moon- r_qi 

    #println("vector from satellite to moon: ", rij_moon)

    #println("distance to moon : ", norm(rij_moon))

    #r_qi - vector that defines the position of the satellite wrt Earth
    #rqj is the position of a planetary body wrt satellite

    #just Earth and Moon
    xdot[4:6] = (-μ_e/(norm(r_qi))^3)*r_qi + μ_m.*((rij_moon/norm(rij_moon)^3)-(rqj_moon/norm(rqj_moon)^3));


    #display((-μ_e/(norm(r_qi))^3)*r_qi)

    #display(μ_m.*((rij_moon/norm(rij_moon)^3)-(rqj_moon/norm(rqj_moon)^3)))
                
    return xdot

end

In [ ]:
#sat_pose = [35677.75392672705, -383227.521788774, -204700.13822902984]
#2024
#moon_pose = [30336.261021197115, -325480.3753119168, -176061.98671653145]

#2018
# sat_pose = [-338899.19758259197, -265393.89399550087, -68933.27991895752]
# moon_pose = [-305665.5295916485, -238568.75389771274, -64082.58948032478]


In [ ]:
# norm(sat_pose - moon_pose)

In [ ]:
# (sat_pose - moon_pose)/norm(sat_pose - moon_pose)

In [ ]:
# moon_pose/norm(moon_pose)

In [ ]:
#these dynamics are all relative to the Earth
# scatter([sat_pose[1]], [sat_pose[2]], [sat_pose[3]], label="Satellite")
# scatter!([moon_pose[1]], [moon_pose[2]], [moon_pose[3]], label= "Moon")
# scatter!([0], [0], [0], label = "Earth")

In [ ]:
#the x is scaled here as well as the t 

function scaled_ephemeris_dynamics(x,t)

    q_original = zeros(eltype(x),3)
    v_original = zeros(eltype(x),3)

    #get to original units (km and s)
    q_original = x[1:3]*l_star 
    v_original = x[4:6]*(l_star/t_star)
    t_original = t*t_star

    x_original = [q_original; v_original]

    #original is in the CR3BP units
    ẋ_original = zeros(eltype(x),6)

    #calculate the original xdot (no scaling)

    ẋ_original = ephemeris_model_EarthMoon(x_original, t_original)
    
    #then scale the output
    v_scaled = ẋ_original[1:3]/(l_star/t_star)
    
    a_scaled = ẋ_original[4:6]/(l_star/(t_star)^2)

    ẋ_scaled = [v_scaled; a_scaled]

    return ẋ_scaled

end

In [ ]:
L_cr = 3.850e5 #in km - distance between centers of m1 and m2
V_cr = 1.025 #in km/s - orbital velocity of m1
T_cr = 2.361e6 #in seconds - orbital period of m1 and m2
time_scale = T_cr/(2*pi)

In [ ]:
L_nasa = 389703
T_nasa = 382981

In [ ]:
#mass paramter for earth moon
μ_earth_moon = 1.215058560962404E-2

In [ ]:
#position of Earth in rotating frame
pose_m1 = [-μ_earth_moon, 0, 0]

In [ ]:
pose_m2 = [1-μ_earth_moon, 0, 0]

In [ ]:
#norm(pose_m2 - x0_test[1:3])*L_cr 

In [ ]:
#seems periodic, not sure if it's correct
# x0_test = [1.1201297302380415,
#  0.0,
#  0.0059396759100811495,
#  0.0,
#  0.17677819141944426,
#  0.0]

#  x0_test_earthcentered = [x0_test[1:3] - pose_m1; x0_test[4:6]]

#  #in cr3bp units 
# T_periodic = 3.414975413662902

# x0_dim = [x0_test_earthcentered[1:3]*L_cr; x0_test_earthcentered[4:6]*(L_cr/T_cr)]

# T_dim = T_periodic*time_scale

In [ ]:
#this is in km and seconds
xtraj_test_initial = [xtraj[1:3,1]; xtraj[4:6,1]/86400]

In [ ]:
#Initial Condtion from xtraj
x0_test = [xtraj_test_initial[1:3] + [μ_earth_moon*l_star, 0, 0]; xtraj_test_initial[4:6]]

#x0_test_earthcentered = [x0_test[1:3] - pose_m1; x0_test[4:6]]

 #in cr3bp units 
T_periodic = T_periodic_n

#x0_dim = [x0_test_earthcentered[1:3]*L_cr; x0_test_earthcentered[4:6]*(L_cr/T_cr)]

T_dim = T_periodic*t_star 

In [ ]:
################################################################################################################
#from https://ssd.jpl.nasa.gov/tools/periodic_orbits.html

#in nasa cr3bp units
#x0_test = [1.1808985497899205E+0, -2.5444988241150091E-26, 1.0295054075242347E-4, 3.3765359485568778E-15, -1.5585631393981156E-1, 5.5263881873244218E-18]
#T_periodic = 3.4155308065628454

#larger orbit
# x0_test = [1.1805068248439281E+0,	-3.3114436259161434E-27,	1.9906874796338979E-2,	3.3672394329658290E-15,	-1.5811356683023692E-1, 2.2436379855191516E-15]
# T_periodic = 3.1503862907153901
# x0_test_earthcentered = [x0_test[1:3] - pose_m1; x0_test[4:6]]


# x0_dim = [x0_test_earthcentered[1:3]*L_nasa; x0_test_earthcentered[4:6]*(L_nasa/T_nasa)]

# T_dim = T_periodic*T_nasa
#################################################################################################################

In [ ]:

# y = 23
# #ECI
# xx = [xtraj[1:3,y] + [μ_earth_moon*l_star, 0, 0]; xtraj[4:6,y]/86400]
# display(xx)
# x_test_ephem = cr3bp_to_ephem(xx, 100)

In [ ]:
# ephem_to_cr3bp(x_test_ephem, 100)

In [ ]:
x_test_ephem = cr3bp_to_ephem(x0_test, 0)

In [ ]:
#this transform gives original x0_dim
#test = ephem_to_cr3bp(x_test_ephem, 0)

In [ ]:
ephemeris_model_EarthMoon(x_test_ephem, 0)

In [ ]:
x_test_ephem_scaled = [x_test_ephem[1:3]/l_star; x_test_ephem[4:6]/(l_star/t_star)]

T_ephem_scaled = T_dim/t_star

In [ ]:
sol = just_dynamics_integrate_scaled(x_test_ephem_scaled, T_ephem_scaled)

all_states = get_state(sol)

In [ ]:
all_states[:,1]

In [ ]:
# T_ephem_scaled

In [ ]:
x_test_ephem_scaled 

In [ ]:
test = [x_test_ephem_scaled[1:3, 1]*l_star; x_test_ephem_scaled[4:6, 1]*(l_star/t_star)]

In [ ]:
ephem_to_cr3bp(test , 0)

In [ ]:
xtraj_test_initial

In [ ]:
# xtraj_i

In [ ]:
# sol.t*t_star

In [ ]:
plot(all_states[1,:]*l_star, all_states[2,:]*l_star, all_states[3,:]*l_star) 
scatter!([all_states[1,1]*l_star], [all_states[2,1]*l_star], [all_states[3,1]*l_star])
scatter!([0], [0], [0])

In [ ]:
#problem: gravity of the moon is much lower than the gravity of Earth 

In [ ]:
using DelimitedFiles
xtraj  = readdlm("xtraj.txt", '\t', Float64, '\n')

#in days
time_steps = readdlm("time.txt", '\t', Float64, '\n')
#period 
#T_periodic_n = 3.414975413662902


T_periodic_n = 3.414975410587678

In [ ]:
#units of km and days

In [ ]:
# xtraj

In [ ]:
# xtraj

In [ ]:
# xtraj_transformed = [xtraj[1:3, :].+ [4677.975459705256, 0, 0]; xtraj[4:6, :]./86400]

In [ ]:
#make sure to divide by 86400 to get into seconds

In [ ]:
# time_steps[2]

In [ ]:
# time_steps

In [ ]:
#transform each of these points into the Earth Moon ephemeris model to see what it looks like

# xtraj_ephemeris = zeros(6,131)

# #x_centered = xtraj_ephemeris 

# for i=1:131

#     xtraj_ephemeris[:,i] = cr3bp_to_ephem(xtraj_transformed[:,i], time_steps[i]*86400)
# end

In [ ]:
# xtraj_ephemeris

In [ ]:
# [all_states[1:3, :].*l_star; all_states[4:6,:].*(l_star/t_star)] 

In [ ]:
#this should be all _states because xtraj_ephemeris is xtraj loaded that was 
#converted 
#xtraj_ephemeris_last = xtraj_ephemeris[:,end]

In [ ]:
# all_states

In [ ]:
# sol.t 

In [ ]:
# all_states 

In [ ]:
#transform all states into cr3bp scaled

#km/s

all_states_cr3bp = zeros(6, size(all_states)[2])

for i=1:size(all_states)[2]

    ephem_scaled = [all_states[1:3,i]*l_star; all_states[4:6,i]*(l_star/t_star)]
    all_states_cr3bp[:,i] = ephem_to_cr3bp(ephem_scaled, sol.t[i]*t_star)
    
end

In [ ]:
#plot(all_states_cr3bp[1,:], all_states_cr3bp[2,:], all_states_cr3bp[3,:])

In [ ]:
# all_states_cr3bp[1:3, 1]

In [ ]:
# all_states_cr3bp[4:6, 1]

In [ ]:
# all_states_cr3bp

In [ ]:
#center around the barycenter

all_states_cr3bp_centered = all_states_cr3bp .- [μ_earth_moon*l_star , 0, 0, 0, 0, 0]

In [ ]:
xtraj./[1,1,1, 86400, 86400, 86400]

In [ ]:
all_states_cr3bp

In [ ]:
xtraj 

In [ ]:
plot(all_states_cr3bp_centered[1,:], all_states_cr3bp_centered[2,:], all_states_cr3bp_centered[3,:])

scatter!([all_states_cr3bp_centered[1,1]], [all_states_cr3bp_centered[2,1]], [all_states_cr3bp_centered[3,1]])

plot!(xtraj[1,:], xtraj[2,:], xtraj[3,:])

scatter!([xtraj[1,1]], [xtraj[2,1]], [xtraj[3,1]]) 

In [ ]:
plot(all_states_cr3bp_centered[1,1:2], all_states_cr3bp_centered[2,1:2], all_states_cr3bp_centered[3,1:2])

scatter!([all_states_cr3bp_centered[1,1]], [all_states_cr3bp_centered[2,1]], [all_states_cr3bp_centered[3,1]])

plot!(xtraj[1,1:50], xtraj[2,1:50], xtraj[3,1:50])

scatter!([xtraj[1,1]], [xtraj[2,1]], [xtraj[3,1]])

In [ ]:
all_states_cr3bp_centered[:,1] - [xtraj[1:3,1]; xtraj[4:6,1]/86400]

In [ ]:
xtraj_ephemeris_last

In [ ]:
xtraj_ephemeris_last_scaled = [xtraj_ephemeris_last[1:3]*l_star; xtraj_ephemeris_last[4:6]*(l_star/t_star)]

In [ ]:
#the units are good here for the velocity. check units overall in the transformation

x_cr3bp = ephem_to_cr3bp(xtraj_ephemeris_last_scaled, 1.2832276238049301e6)

In [ ]:
sol.t*t_star 

In [ ]:
x_cr3bp_normalized = [x_cr3bp[1:3]/l_star; x_cr3bp[4:6]/(l_star/t_star)]

In [ ]:
all_states_cr3bp[:,end]

In [ ]:
μ_earth_moon

In [ ]:
xtraj

In [ ]:
time_steps*time_scale

In [ ]:
#center at the barycenter
x_cr3bp_normalized_centered = x_cr3bp_normalized-[μ_earth_moon, 0,0,0,0,0]


In [ ]:
# x0_test = [1.1201297302380415,
#  0.0,
#  0.0059396759100811495,
#  0.0,
#  0.17677819141944426,
#  0.0]

In [ ]:
#this is periodic in the Earth centered J2000 frame
#now need to convert the final state of the J200 frame to a state in the CR3BP

In [ ]:
#T_dim

In [ ]:
moon_orbit = zeros(6, 1283227)


for i=1:1283227

    moon_orbit[:,i] = spkezr("moon",et+i,"J2000","NONE","earth")[1]

end

In [ ]:
plot(moon_orbit[1,:], moon_orbit[2,:], moon_orbit[3,:])

scatter!([0], [0], [0])

In [ ]:
#I believe there is something wrong with the epoch...

In [ ]:
#plot(xtraj_ephemeris[1,:], xtraj_ephemeris[2,:], xtraj_ephemeris[3,:])

plot(all_states[1,:]*l_star, all_states[2,:]*l_star, all_states[3,:]*l_star) 
scatter!([all_states[1,1]*l_star], [all_states[2,1]*l_star], [all_states[3,1]*l_star])
scatter!([0], [0], [0])
plot!(moon_orbit[1,:], moon_orbit[2,:], moon_orbit[3,:])

#this is relative to the Earth 

In [ ]:
time_steps

In [ ]:
time_steps

In [ ]:
#μ_earth_moon*l_star 

In [ ]:
xtraj_transformed = [xtraj[1:3, :].+ [μ_earth_moon*l_star , 0, 0]; xtraj[4:6, :]./86400]

In [ ]:
#transform each of these points into the Earth Moon ephemeris model to see what it looks like

xtraj_ephemeris = zeros(6,131)

#x_centered = xtraj_ephemeris 

for i=1:131

    xtraj_ephemeris[:,i] = cr3bp_to_ephem(xtraj_transformed[:,i], time_steps[i]*86400)
end

In [ ]:
xtraj_ephemeris

In [ ]:
plot(xtraj_ephemeris[1,:], xtraj_ephemeris[2,:], xtraj_ephemeris[3,:])

plot!(all_states[1,:]*l_star, all_states[2,:]*l_star, all_states[3,:]*l_star) 

In [ ]:
xtraj_ephemeris

In [ ]:
[all_states[1:3, :]*l_star; all_states[4:6, :]*(l_star/t_star)][:,end]

In [ ]:
#convert both back to x_cr3bp

In [ ]:
xtraj_ephemeris

In [ ]:
all_states_scaled = [all_states[1:3, :]*l_star; all_states[4:6, :]*(l_star/t_star)]

In [ ]:
sol.t*t_star

In [ ]:
time_steps*86400

In [ ]:
#11 seconds vs 6409 seconds? 

In [ ]:
all_states_cr3bp = zeros(6, 316)


for i=1:316
    all_states_cr3bp[:,i] = ephem_to_cr3bp(all_states_scaled[:,i], sol.t[i]*t_star)
end

In [ ]:
xtraj_cr3bp = zeros(6,131)

#x_centered = xtraj_ephemeris 

for i=1:131

    xtraj_cr3bp[:,i] = ephem_to_cr3bp(xtraj_ephemeris[:,i], time_steps[i]*86400)
end

In [ ]:
μ_earth_moon*l_star 

In [ ]:
xtraj_cr3bp

In [ ]:
xtraj_transformed

In [ ]:
all_states_cr3bp

In [ ]:
all_states_cr3bp[:,1] - xtraj_cr3bp[:,1]

In [ ]:
all_states_cr3bp

In [ ]:
xtraj_cr3bp

In [ ]:
#plot(xtraj_transformed[1,:], xtraj_transformed[2,:], xtraj_transformed[3,:])

plot(xtraj_cr3bp[1,1:40], xtraj_cr3bp[2,1:40], xtraj_cr3bp[3,1:40])

plot!(all_states_cr3bp[1,1:2], all_states_cr3bp[2,1:2], all_states_cr3bp[3,1:2])

scatter!([all_states_cr3bp[1,1]], [all_states_cr3bp[2,1]], [all_states_cr3bp[3,1]])

scatter!([xtraj_cr3bp[1,1]], [xtraj_cr3bp[2,1]], [xtraj_cr3bp[3,1]])

In [ ]:
# #transform all states into cr3bp scaled

# all_states_cr3bp = zeros(6, size(all_states)[2])


# for i=1:size(all_states)[2]

#     ephem_scaled = [all_states[1:3,i]*l_star; all_states[4:6,i]*(l_star/t_star)]
#     all_states_cr3bp[:,i] = ephem_to_cr3bp(ephem_scaled, sol.t[i]*t_star)
    
# end

In [ ]:
#tested on a leo orbit and dynamics work fine

#something wrong with the ephemeris to cr3bp conversion? 

In [ ]:
#rk4 to integrate the ephemeris equation

In [ ]:
function RK4_integrator(x, t)

    Δt = 1
    f1 = ephemeris_model_EarthMoon(x, t)
    f2 = ephemeris_model_EarthMoon(x+0.5*Δt*f1, t + Δt/2)
    f3 = ephemeris_model_EarthMoon(x+0.5*Δt*f2, t + Δt/2)
    f4 = ephemeris_model_EarthMoon(x+ Δt*f3, t+Δt)

    xnext = x + (Δt/6.0)*(f1+2*f2+2*f3+f4)

    return xnext

end

In [ ]:
all_states_rk4 = zeros(6, 1000000)

all_states_rk4[:,1] = x_test_ephem

for i=1:1000000-1
    all_states_rk4[:,i+1] = RK4_integrator(all_states_rk4[:,i], i)

end

In [ ]:
all_states_rk4

In [ ]:
plot(all_states_rk4[1,:], all_states_rk4[2,:], all_states_rk4[3,:])

plot!(all_states[1,:]*l_star, all_states[2,:]*l_star, all_states[3,:]*l_star) 

In [ ]:
all_states_rk4

In [ ]:
x_test_ephem

In [ ]:
#potential error is with the transformation matrix. the part 
#corresponding to the velocities. Make sure to write out the
#derivation